In [1]:
import numpy as np
import neuralNetwork 
from os.path  import join
from datasetLoad import MnistDataloader

28


In [2]:
input_path = r'C:\programs\projects\NN\input'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')


mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

x_train = np.array(x_train)
x_test = np.array(x_test)

# print(x_test[1],y_test[1])

In [3]:
# Flatten the training and test images
x_train_flat = x_train.reshape(x_train.shape[0], -1)  
x_test_flat = x_test.reshape(x_test.shape[0], -1)     

testInputs = [784 ,3,784,16,'relu',16,16,'relu',16,10,'sigmoid']
model = neuralNetwork.ModelTest(testInputs)

'''
input parameters = 784
number of layers = 3

layer 1:
number of input: 784
number of neurons: 16
Enter the activation funtion: relu

layer 2:
number of input: 16
number of neurons: 16
Enter the activation funtion: relu

layer 3:
number of input: 16
number of neurons: 10
Enter the activation funtion: sigmoid
'''

'\ninput parameters = 784\nnumber of layers = 3\n\nlayer 1:\nnumber of input: 784\nnumber of neurons: 16\nEnter the activation funtion: relu\n\nlayer 2:\nnumber of input: 16\nnumber of neurons: 16\nEnter the activation funtion: relu\n\nlayer 3:\nnumber of input: 16\nnumber of neurons: 10\nEnter the activation funtion: sigmoid\n'

In [4]:
for x, y in zip(x_train_flat, y_train):
    x = np.array([x])
    model.train(x, y)


here
shapeofIN = (1, 784), shapeofWI = (784, 16)
shapeofIN = (1, 16), shapeofWI = (16, 16)
shapeofIN = (1, 16), shapeofWI = (16, 10)
wei = (16, 16)
new = [[   0.         1512.8856346    83.25098562  607.37425209    0.
   433.48998939    8.90985075 1570.58619258  130.14747408 2148.89883487
     0.         1177.33412235  872.31690416  393.50758381  802.58227331
  3529.86465108]]
here
shapeofIN = (1, 784), shapeofWI = (784, 16)
shapeofIN = (1, 16), shapeofWI = (784, 16)


c:\programs\projects\NN\neuralNetwork.py:34: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-values))


ValueError: shapes (1,16) and (784,16) not aligned: 16 (dim 1) != 784 (dim 0)

In [ ]:
a = np.array([x_train_flat[0]])
model.pridict(a)

[[ 312.70795421    0.          659.66035991    0.            0.
   202.32734049    0.            0.            0.            0.
   796.61605486 1666.14526113 1366.73697261  405.05207695   63.8154843
     0.        ]]
[[1244.67081574    0.           50.4632191     0.            0.
     0.          389.93838025    0.            0.          791.08213188
  1399.00574794    0.            0.          261.57572019  914.36981593
     0.        ]]
[[0.00000000e+000 1.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 1.70889831e-309 0.00000000e+000
  0.00000000e+000 0.00000000e+000]]


{'label': 9, 'confidence': 0.0}